In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
import matplotlib.patches as mpatches
plt.style.use('ggplot')

In [2]:
def datasets():
    #Training Dataset
    train = pd.read_csv("../input/pred-student-perf-without-competiton-env/train.csv")
    #Importing Training Labels
    labels = pd.read_csv("../input/pred-student-perf-without-competiton-env/train_labels.csv")
    #Test Dataset
    test = pd.read_csv("../input/pred-student-perf-without-competiton-env/test.csv")
    return train, labels, test

In [3]:
train_df, labels_df, test_df = datasets()

FileNotFoundError: ignored

In [ ]:
test_df['session'] = test_df.session_id.apply(lambda x: int(x.split('_')[0]) )
test_df['q'] = test_df.session_id.apply(lambda x: int(x.split('_')[-1][1:]) )
print( test_df.shape )
test_df.head()

Note some codes were redundant so i have removed them from inside


In [ ]:
train_df.head(10)

In [ ]:
train_df = train_df.sort_values(['session_id','elapsed_time'])
print(train_df)

In [ ]:
train_df.describe()

In [ ]:
print(train_df.dtypes)

In [ ]:
no_rows, no_columns = train_df.shape
print(f"Training Data Rows and Columns Information \nNumber of Rows: {no_rows}\nNumber of Columns: {no_columns}")

In [ ]:
print("nan values in Training Data", train_df.isna().sum().sort_values(ascending=True))

In [ ]:
events = train_df["event_name"].unique()
print("All listed Events in Training Data are:", events)

In [ ]:
eve = train_df["event_name"].value_counts().plot(kind='barh', title = "Event Names", color = 'green')
eve.set_xlabel("Number of Events")
eve.set_ylabel("Name of Events")
plt.show()

In [ ]:
plt.figure(figsize = (5, 5))
eve = train_df["name"].value_counts().plot(kind='bar', title = "Names", color = 'orange')
eve.set_xlabel("Name")
eve.set_ylabel("Occurence")
plt.show()

In [ ]:
names = train_df["name"].unique()
print("Name in Training Data are:", names)
names = train_df["name"].value_counts().sort_values(ascending=True)
print(names)

In [ ]:
lev = train_df["level"].unique()
print("Levels in Training Data are:", lev)
lev = train_df["level"].value_counts().sort_values(ascending=True)
print(lev)

In [ ]:
plt.figure(figsize = (8, 8))
lev = train_df["level"].value_counts().plot(kind='barh', title = "Levels", color = 'blue')
lev.set_xlabel("Occurence")
lev.set_ylabel("Level")
plt.show()

In [ ]:
tim = train_df["level_group"].unique()
print("Name in Training Data are:", tim)
tim = train_df["level_group"].value_counts()
print(tim)

In [ ]:
plt.figure(figsize = (5, 5))
plt.style.use('bmh')
lev = train_df["level_group"].value_counts().plot(kind='bar', title = "Level_group", color = 'red')
lev.set_xlabel("Level_group")
lev.set_ylabel("Occurence")
plt.show()

In [ ]:
f_qid = train_df["elapsed_time"].unique()
print("Name in Training Data are:", f_qid)
f_qid = train_df["elapsed_time"].value_counts().sort_index(ascending=True)
print(f_qid)

In [ ]:
from sklearn import preprocessing

normalized_arr = preprocessing.normalize([train_df.elapsed_time])
print(normalized_arr)

In [ ]:
plt.figure(figsize = (7, 7))
sns.distplot(normalized_arr)

plt.grid()
plt.title("Elapsed time - distribution plot")
plt.show()

In [ ]:
plt.figure(figsize=(5, 5))

session_id = train_df["session_id"]
print(session_id)
sns.distplot(session_id, kde=True)
plt.xlabel("session ID", fontsize=12)
plt.ylabel("Count", fontsize=12)
plt.title("Session ID", fontsize=16)


In [ ]:
train_df.dropna(["hover_duration". "page", "music", "fullscreen", "hq"], axis =1 , inplace = True)


In [ ]:
train_df.drop_duplicates(["room_coor_x", "room_coor_y", "screen_coor_x" ,"screen_coor_y"], inplace = True)

In [ ]:
for col in train_df.columns:
  if train_df[col].dtype != "object":
    train_df[col] = data[col].fillna(data[col].mean())

In [ ]:
for col in data.columns:
  if train_df[col].dtype =="object":
    train_df[col] = data[col].fillna(data[col].mode())

In [ ]:
plt.figure(figsize= (14,12))
ax = sns.countplot(x = train_df["event_name"], data = train_df)


In [ ]:
def feature_engineer(train_df):
    
    df = []
    for c in categorical_values:
        temp_feature = train_df.groupby(['session_id','level_group'])[c].agg('nunique')
        temp_feature.name = temp_feature.name + '_nunique'
        dfs.append(temp_feature)
    for c in numerical_values:
        temp_feature = train_df.groupby(['session_id','level_group'])[c].agg('median')
        temp_feature.name = temp_feature.name + '_median'
        train_df.append(temp_feature)
    for c in numerical_values:
        temp_feature = train_df.groupby(['session_id','level_group'])[c].agg('std')
        temp_feature.name = temp_feature.name + '_std'
        train_df.append(temp_feature)
    for c in numerical_values: 
        train[c] = (train_df.event_name == c).astype('int8')
    for c in numerical_values + ['elapsed_time']:
        temp_feature = train_df.groupby(['session_id','level_group'])[c].agg('sum')
        temp_feature.name = temp_feature.name + '_sum'
        train_df.append(temp_feature)
    train_df.drop(events_loc,axis=1, inplace = True)
        
    train_df = pd.concat(dfs,axis=1)
    train_df = train_df.fillna(-1)
    train_df = train_df.reset_index()
    train_df = train_df.set_index('session_id')
    return train_df

In [ ]:
FEATURES = [c for c in train_df.columns if c != 'level_group']
print('We will train with', len(FEATURES) ,'features')
ALL_USERS = train_df.index.unique()
print('We will train with', len(ALL_USERS) ,'users info')

In [ ]:
gkf = GroupKFold(n_splits=20)
oof = pd.DataFrame(data=np.zeros((len(ALL_USERS),18)), index=ALL_USERS)
models = {}


for i, (train_index, test_index) in enumerate(gkf.split(X=train_df, groups=train_df.index)):
    print('#'*25)
    print('### Fold',i+1)
    print('#'*25)
    
    xgb_params = {
    'objective' : 'binary:logistic',
    'eval_metric':'logloss',
    'learning_rate': 0.005,
    'max_depth': 4,
    'n_estimators': 1500,
    'early_stopping_rounds': 50,
    'tree_method':'hist',
    'subsample':0.8,
    'colsample_bytree': 0.4,
    'use_label_encoder' : False}
    
    # ITERATE THRU QUESTIONS 1 THRU 18
    for t in range(1,19):
        
        # USE THIS TRAIN DATA WITH THESE QUESTIONS
        if t<=3: grp = '0-4'
        elif t<=13: grp = '5-12'
        elif t<=22: grp = '13-22'
            
        # TRAIN DATA
        train_x = train_df.iloc[train_index]
        train_x = train_x.loc[train_x.level_group == grp]
        train_users = train_x.index.values
        train_y = targets.loc[targets.q==t].set_index('session').loc[train_users]
        
        # VALID DATA
        valid_x = df.iloc[test_index]
        valid_x = valid_x.loc[valid_x.level_group == grp]
        valid_users = valid_x.index.values
        valid_y = targets.loc[targets.q==t].set_index('session').loc[valid_users]
        
        # TRAIN MODEL        
        clf =  XGBClassifier(**xgb_params)
        clf.fit(train_x[FEATURES].astype('float32'), train_y['correct'],
                eval_set=[ (valid_x[FEATURES].astype('float32'), valid_y['correct']) ],
                verbose=0)
        print(f'{t}({clf.best_ntree_limit}), ',end='')
        
        # SAVE MODEL, PREDICT VALID OOF
        models[f'{grp}_{t}'] = clf
        oof.loc[valid_users, t-1] = clf.predict_proba(valid_x[FEATURES].astype('float32'))[:,1]
        
    print()

In [ ]:
scores = []; thresholds = []
best_score = 0; best_threshold = 0

for threshold in np.arange(0.4,0.81,0.01):
    print(f'{threshold:.02f}, ',end='')
    preds = (oof.values.reshape((-1))>threshold).astype('int')
    m = f1_score(true.values.reshape((-1)), preds, average='macro')   
    scores.append(m)
    thresholds.append(threshold)
    if m>best_score:
        best_score = m
        best_threshold = threshold

In [ ]:
import matplotlib.pyplot as plt

# PLOT THRESHOLD VS. F1_SCORE
plt.figure(figsize=(20,5))
plt.plot(thresholds,scores,'-o',color='blue')
plt.scatter([best_threshold], [best_score], color='blue', s=300, alpha=1)
plt.xlabel('Threshold',size=14)
plt.ylabel('Validation F1 Score',size=14)
plt.title(f'Threshold vs. F1_Score with Best F1_Score = {best_score:.3f} at Best Threshold = {best_threshold:.3}',size=18)
plt.show()


In [ ]:
print('When using optimal threshold...')
for k in range(18):
        
    # COMPUTE F1 SCORE PER QUESTION
    m = f1_score(true[k].values, (oof[k].values>best_threshold).astype('int'), average='macro')
    print(f'Q{k}: F1 =',m)
    
# COMPUTE F1 SCORE OVERALL
m = f1_score(true.values.reshape((-1)), (oof.values.reshape((-1))>best_threshold).astype('int'), average='macro')
print('==> Overall F1 =',m)